This example aims to show different Euler angles combinations to position a camera coordinate system

In [ ]:
%matplotlib notebook
import os
import screeninfo
import cv2 as cv
import numpy as np
import math
from vcd import core
from vcd import types
from vcd import utils
from vcd import draw
from vcd import scl

import matplotlib.pyplot as plt

def setup():
    vcd = core.VCD()
    vcd.add_coordinate_system("vehicle-iso8855", cs_type=types.CoordinateSystemType.local_cs)
    
    # Let's build the cameras
    img_width_px = 1280
    img_height_px = 966
    
    ################################
    # CAM_FRONT Yaw-Pitch-Roll
    ################################
     # Intrinsics
    # ------------------------------
    # This matrix converts from scs to ics
    d_1x4 = np.array([333.437012, 0.307729989, 2.4235599, 11.0495005])
    cX = -0.302159995
    cY = -3.44617009
    
    # Extrinsics (1/2): Rotation using yaw(Z), pitch(Y), roll(X). This is rotation of SCS wrt LCS
    # Common understanding is that a coordinate system is transformed applying yaw->pitch->roll order
    # The Rotation matrix we build here defines a Pose, that is, an alias intrinsic rotation.
    # Consequently, an alias intrinsic rotation is obtained using the inverse order of rotation.
    # For instance, R=RzRyRx mathematically means Rx(roll) is applied first, then Ry(pitch), then Rz (yaw)
    # The alibi transformation is obtained inverting the pose, and then the 3D points are rotated in the yaw->pitch->roll order
    yaw_rad = (0.0 * np.pi) / 180.0
    pitch_rad = (10.0 * np.pi) / 180.0    
    roll_rad = (0.0 * np.pi) / 180.0
    
    C_scs_wrt_lcs = np.array([[2.3],  # frontal part of the car
                      [0.5],  # centered in the symmetry axis of the car, slightly at the left
                      [1.3]])  # at some height over the ground
    
    R_scs_wrt_lcs = utils.euler2R([yaw_rad, pitch_rad, roll_rad])  # default is ZYX, R=RzRyRx, which is alias Z->Y->X    
    print(R_scs_wrt_lcs)
    P_scs_wrt_lcs = utils.create_pose(R_scs_wrt_lcs, C_scs_wrt_lcs)    
    print(P_scs_wrt_lcs)
    
    # The transform from LCS points to SCS points is the inverse of the pose
    T_lcs_to_scs = utils.inv(P_scs_wrt_lcs)
        
    # Extrinsics (2/2): Additional rotation, because usually cameras scs are (x-to-right, y-down, z-to-front)
    # to match better with ics (x-to-right, y-bottom); while the lcs is (x-to-front, y-to-left, z-up)
    T_image = np.array([[0.0, -1.0, 0.0, 0.0],
                                [0.0, 0.0, -1.0, 0.0],
                                [1.0, 0.0, 0.0, 0.0],
                                [0.0, 0.0, 0.0, 1.0]])
     # Now we can compose the final transform
    T_lcs_to_scs = T_image.dot(T_lcs_to_scs)
    P_scs_wrt_lcs = utils.inv(T_lcs_to_scs)
    
    vcd.add_stream(stream_name="CAM_FRONT_YPR",
                   uri="",
                   description="Virtual camera",
                   stream_type=core.StreamType.camera)
    vcd.add_stream_properties(stream_name="CAM_FRONT_YPR",
                              intrinsics=types.IntrinsicsFisheye(
                                  width_px=img_width_px,
                                  height_px=img_height_px,
                                  lens_coeffs_1x4=list(d_1x4.flatten()),
                                  center_x=cX,
                                  center_y=cY,
                                  aspect_ratio=1.0
                              )
                              )
    vcd.add_coordinate_system("CAM_FRONT_YPR", cs_type=types.CoordinateSystemType.sensor_cs,
                              parent_name="vehicle-iso8855",
                              pose_wrt_parent=types.PoseData(
                                  val=list(P_scs_wrt_lcs.flatten()),
                                  type=types.TransformDataType.matrix_4x4)
                              )
    
    ################################
    # CAM_FRONT Z1-X-Z2
    ################################
     # Intrinsics
    # ------------------------------
    # This matrix converts from scs to ics
    d_1x4 = np.array([333.437012, 0.307729989, 2.4235599, 11.0495005])
    cX = -0.302159995
    cY = -3.44617009
    
    # Extrinsics
    # ------------------------------
    #X-Z1-Z2 provided wrt z-down x-back y-left
    # to be applied z1xz2
    x1 = np.pi/2 #(91.4561996 * np.pi) / 180.0
    z1 = -np.pi/2 #(-90.9681015 * np.pi) / 180.0
    z2 = 0 #(-3.01329994 * np.pi) / 180.0
    
    C_scs_wrt_lcs = np.array([[2.3],  # frontal part of the car
                      [-0.5],  # centered in the symmetry axis of the car, slightly at the right
                      [1.3]])  # at some height over the ground
    
    R = utils.euler2R([z1, x1, z2], seq=utils.EulerSeq.ZXZ)
    
    # To transform from ISO8855 convention (x-front, y-left, z-up) to Z1XZ2 convention, 180º rotation in y-axis is needed
    R_iso =  utils.euler2R([0, np.pi, 0], seq=utils.EulerSeq.ZYX) # i.e. applied in order Z->Y->X 
    
    R = R_iso.dot(R)  # R_iso is at the left because we are still building alias intrinsic rotations, and thus, the
    # order needs to be inverse to the mental order needed to transform the coordinate system    

    # We can now compose the Pose
    P_scs_wrt_lcs = utils.create_pose(R, C_scs_wrt_lcs)
    
    vcd.add_stream(stream_name="CAM_FRONT_Z1XZ2",
                   uri="",
                   description="Virtual camera",
                   stream_type=core.StreamType.camera)
    vcd.add_stream_properties(stream_name="CAM_FRONT_Z1XZ2",
                              intrinsics=types.IntrinsicsFisheye(
                                  width_px=img_width_px,
                                  height_px=img_height_px,
                                  lens_coeffs_1x4=list(d_1x4.flatten()),
                                  center_x=cX,
                                  center_y=cY,
                                  aspect_ratio=1.0
                              )
                              )
    vcd.add_coordinate_system("CAM_FRONT_Z1XZ2", cs_type=types.CoordinateSystemType.sensor_cs,
                              parent_name="vehicle-iso8855",
                              pose_wrt_parent=types.PoseData(
                                  val=list(P_scs_wrt_lcs.flatten()),
                                  type=types.TransformDataType.matrix_4x4)
                              )
    
    return vcd
    
    
# Run program
vcd = setup()
scene = scl.Scene(vcd)
setupViewer = draw.SetupViewer(scene, "vehicle-iso8855")
fig1 = setupViewer.plot_setup()
plt.show()
print('Process finished')

    
    